# Production

Необходимые библиотеки и модули для работы алгоритма

In [ ]:
%pip install tinvest

In [ ]:
import tinvest
import time
import datetime
from math import sqrt

Функция для нахождения процента изменения от исходной цены

In [ ]:
def calc_procent(new_price, last_price):
    return ((new_price - last_price)/last_price) * 100

Функция для нахождения абсолютной разницы от исходной цены

In [ ]:
def calc_diff(new_price, last_price):
    return round((new_price - last_price), 2)

Функция для решения частного случая работы алгоритма (при выкупе нескольких заявок подряд на старте)

In [ ]:
def findCount(martingail, count_lots, clots):
    d = (count_lots - 1/2*martingail)**2 - 2 * martingail * -clots
    x_1 = (-(count_lots - 1/2 * martingail) + sqrt(d))/martingail
    x_2 = (-(count_lots - 1/2 * martingail) - sqrt(d))/martingail
    return int(x_1 if x_1 >= 0 else x_2)

Формула для функции findCount

$$ 
l\cdot n\ +\ m\cdot\frac{1}{2}\cdot\left(n-1\right)\cdot n\ -\ s=0
$$

m - martingail

l - lots per order

s - summa

n - count - ? (возврат данного значения из функции findCount)

Функция формирования сетки для 3 случаев:
1. При старте запуска алгоритма (от начально заданной цены)
2. Когда цена идет вверх на 4% и выше
3. Когда цена идет вниз на 4% и ниже

In [ ]:
def create_grid(first_price, diff_grid, cshorts, clongs, new_grid=False, up = False):
    if new_grid and up:
        grid = [round(first_price + (2*diff_grid * i),2) for i in range(1, cshorts + 1)]
        grid.extend([round(first_price - (0.15 * i),2) for i in range(1, 2)])
        grid.extend([round(first_price - (diff_grid * i),2) for i in range(4, clongs + 2 + 1)]) 
    elif new_grid and not up:
        grid = [round(first_price + (0.2 * i),2) for i in range(1, 2)]
        grid.extend([round(first_price + (diff_grid * i),2) for i in range(5, cshorts + 3 + 1)])
        grid.extend([round(first_price - (diff_grid * i),2) for i in range(1, clongs + 1)])
    else:
        grid = [round(first_price + (diff_grid * i),2) for i in range(1, cshorts + 1)]
        grid.extend([round(first_price - (diff_grid * i),2) for i in range(1, clongs + 1)])
    return grid

Функция сброса параметров алгоритма при перестроении сетки. Параметры, которые сохраняют информации об открытых шортах/лонгах на данный момент.

In [ ]:
def reset_temp_params():
    global orders_short, orders_long, count_short, count_long, dict_short_orders, dict_long_orders, count, clots
    orders_short = []
    orders_long = []
    count_short = 0
    count_long = 0
    dict_short_orders = {}
    dict_long_orders = {}
    count = 0
    clots = 0

Токен из личного кабинета для связи с брокером через API и id брокерского счета

In [ ]:
token = None # FIXMME
account_id = None# FIXME

Настройки функционала для работы с API

In [ ]:
client = tinvest.SyncClient(token, use_sandbox=False)

portfolio_api = tinvest.PortfolioApi(client)
accounts_api = tinvest.UserApi(client)
market_api = tinvest.MarketApi(client)
orders_api = tinvest.OrdersApi(client)
operations_api = tinvest.OperationsApi(client)

### Динамические параметры.

- **close_diff** - take profit (значение, при котором закрываем сделку)
- **first_price** - первоначальная цена
- **count_lots_short** - количество лотов при открытии одной сделки в шорт
- **count_lots_long** - количество лотов при открытии одной сделки в лонг
- **martingail** - значение мартингейл (только арифметическая прогрессия)
- **diff_grid** - абсолютное значение деления сетки
- **cshorts = 0 // count_lots_short** - вместо нуля подставить общее количество доступных лотов в шорт (определяет размер сетки в шорт)
- **clongs = 0 // count_lots_long** - вместо нуля подставить общее количество доступных лотов в лонг (определяет размер сетки в лонг)
- **last_day_close_procent** - процент закрытия последнего дня (относительно этого параметра считается процент изменения для перенастройки сетки и сброса всех статических параметром)
- **timer** - через какое время делать запрос (50 запросов в минуту - максимум)
- **count_new_grid** - сколько раз перестраивать сетку (0 - если ни разу)
- **let** - True - если разрешить совершать сделки, False - если нужно видеть только информацию из стакана во время работы алгоритма
- **new_grid_lots_short** - количество лотов в большой шорт
- **new_grid_lots_long** - количество лотов в большой лонг

Опциональные параметры

- oborot - оборотка (ставить не менее 3000 - в долларах, когда необходимо учитывать прибыль с комиссией 0.025%)
- marga - текущая чистая прибыль (с учетом комиссии)

In [ ]:
close_diff = 0.03
first_price = 10.1
count_lots_short = 50
count_lots_long = 100
martingail = 0
diff_grid = 0.05
cshorts = 400 // count_lots_short
clongs = 700 // count_lots_long
timer = 2
let = False

# для новых сеток
count_new_grid = 0
last_day_close_procent = 4
new_grid_lots_short = 150
new_grid_lots_long = 300

# опциональные для статистики
oborot = 4000
marga = 0

Количество делений в шорт

In [ ]:
cshorts

8

Количество делений в лонг

In [ ]:
clongs

7

### Статические параметры. (не изменяемые)

In [ ]:
orders_short = []
orders_long = []
count_short = 0
count_long = 0
dict_short_orders = {}
dict_long_orders = {}
count = 0
clots = 0
temp_price = 0

grid = create_grid(first_price, diff_grid, cshorts, clongs)

try:
    figi_rrc = market_api.market_search_by_ticker_get(ticker='RRC').json()['payload']['instruments'][0]['figi']
except Exception as e:
    print(e.args)  # tinvest.Error

Сформированная сетка

In [ ]:
grid

[10.15,
 10.2,
 10.25,
 10.3,
 10.35,
 10.4,
 10.45,
 10.5,
 10.05,
 10.0,
 9.95,
 9.9,
 9.85,
 9.8,
 9.75]

Основной блок работы алгоритма

In [ ]:
try:
    while True:
        try:
            orderbook = market_api.market_orderbook_get(figi=figi_rrc, depth=1).json()
        except Exception as e:
            print('Ошибка получения Orderbook: {}'.format(e.args))
            time.sleep(timer)
            continue
        
        ask = orderbook['payload']['asks'][0]['price']
        bid = orderbook['payload']['bids'][0]['price']

        new_price = bid if bid >= first_price else ask

        print('Цена текущая:', str(new_price), '| Цена начальная:', str(first_price))
        print('Ask:', str(ask), '| Bid:', str(bid))
        
        procent = calc_procent(new_price, first_price)
        
        if count_new_grid != 0:
            if abs(procent) >= last_day_close_procent and procent > 0:
                if orders_short:
                    for order in orders_short:
                        if dict_short_orders.get(order):
                            body_order = tinvest.schemas.MarketOrderRequest(lots=dict_short_orders[order], operation=tinvest.schemas.OperationType.buy)
                            orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order)
                    reset_temp_params()
                    body_order = tinvest.schemas.MarketOrderRequest(lots=new_grid_lots_long, operation=tinvest.schemas.OperationType.buy)
                    orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order)
                    temp_price = first_price
                    let = False
                    print('ЗАКРЫВАЕМ ВСЕ ШОРТЫ, ВА-БАНК НА ЛОНГ')
                    continue
                if abs(procent) >= (last_day_close_procent + 1) and procent > 0 and not let:
                    body_order = tinvest.schemas.MarketOrderRequest(lots=new_grid_lots_long, operation=tinvest.schemas.OperationType.sell)
                    orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order)
                    first_price = new_price
                    count_new_grid -= 1
                    let = True
                    grid = create_grid(first_price, 0.05, cshorts, clongs, True, True)
                    print('ЗАКРЫВАЕМ ВА-БАНК ЛОНГ, first_price: {}'.format(first_price))
                    continue

            if not let and abs(procent) <= (last_day_close_procent - 1) and procent > 0:
                body_order = tinvest.schemas.MarketOrderRequest(lots=new_grid_lots_long, operation=tinvest.schemas.OperationType.sell)
                orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order)
                first_price = temp_price
                reset_temp_params()
                grid = create_grid(first_price, diff_grid, cshorts, clongs)
                let = True
                print('СТОП-ЛОСС ВА-БАНК ЛОНГ, first_price: {}'.format(first_price))
                continue


            if abs(procent) >= last_day_close_procent and procent < 0:
                if orders_long:
                    for order in orders_long:
                        if dict_long_orders.get(order):
                            body_order = tinvest.schemas.MarketOrderRequest(lots=dict_long_orders[order], operation=tinvest.schemas.OperationType.sell)
                            orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order)
                    reset_temp_params()
                    body_order = tinvest.schemas.MarketOrderRequest(lots=new_grid_lots_short, operation=tinvest.schemas.OperationType.sell)
                    orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order)
                    temp_price = first_price
                    let = False
                    print('ЗАКРЫВАЕМ ВСЕ ЛОНГИ, ВА-БАНК НА ШОРТ')
                    continue
                if abs(procent) >= (last_day_close_procent + 1) and procent < 0 and not let:
                    body_order = tinvest.schemas.MarketOrderRequest(lots=new_grid_lots_short, operation=tinvest.schemas.OperationType.buy)
                    orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order)
                    first_price = new_price
                    count_new_grid -= 1
                    let = True
                    grid = create_grid(first_price, 0.05, cshorts, clongs, True, False)
                    print('ЗАКРЫВАЕМ ВА-БАНК ШОРТ, first_price: {}'.format(first_price))
                    continue

            if not let and abs(procent) <= (last_day_close_procent - 1) and procent < 0:
                body_order = tinvest.schemas.MarketOrderRequest(lots=new_grid_lots_short, operation=tinvest.schemas.OperationType.buy)
                orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order)
                first_price = temp_price
                reset_temp_params()
                grid = create_grid(first_price, diff_grid, cshorts, clongs)
                let = True
                print('СТОП-ЛОСС ВА-БАНК ШОРТ, first_price: {}'.format(first_price))
                continue
            
        if let:
            if procent > 0: # SHORT
                if new_price not in orders_short:
                    for grid_order in grid[:cshorts]:
                        if new_price >= grid_order and ((new_price > max(orders_short) and grid_order > max(orders_short)) if orders_short else True):
                            body_order = tinvest.schemas.MarketOrderRequest(lots=(count_lots_short + int(martingail*count)), operation=tinvest.schemas.OperationType.sell)
                            try:
                                count_short += orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order).json()['payload']['executedLots']
                            except Exception as e:
                                print('Exception Short open: {}'.format(e.args))
                                time.sleep(timer)
                                continue
                            oborot += (count_lots_short + int(martingail*count)) * new_price
                            marga -= (count_lots_short + int(martingail*count)) * new_price * 0.0005 if oborot <= 200000 else (count_lots_short + int(martingail*count)) * new_price * 0.00025 #комиссия
                            count += 1
                    if count_short != 0:
                        print('Открываем ШОРТ: {} | Кол-во заявок: {} | Всего заявок: {} | Оборотка: {} | Профит: {}'.format(new_price, count_short, count, oborot, marga))
                        print()
                        dict_short_orders[new_price] = count_short
                        orders_short.append(new_price)
                        count_short = 0
            elif procent < 0: # LONG
                if new_price not in orders_long:
                    for grid_order in grid[-clongs:]:
                        if grid_order >= new_price and ((new_price < min(orders_long) and grid_order < min(orders_long)) if orders_long else True):
                            body_order = tinvest.schemas.MarketOrderRequest(lots=(count_lots_long + int(martingail*count)), operation=tinvest.schemas.OperationType.buy)
                            try:
                                count_long += orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order).json()['payload']['executedLots']
                            except Exception as e:
                                print('Exception Long open: {}'.format(e.args))
                                time.sleep(timer)
                                continue
                            oborot += (count_lots_long + int(martingail*count)) * new_price
                            marga -= (count_lots_long + int(martingail*count)) * new_price * 0.0005 if oborot <= 200000 else (count_lots_long + int(martingail*count)) * new_price * 0.00025 #комиссия
                            count += 1
                    if count_long != 0:
                        print('Открываем ЛОНГ: {} | Кол-во заявок: {} | Всего заявок: {} | Оборотка: {} | Профит: {}'.format(new_price, count_long, count, oborot, marga))
                        print()
                        dict_long_orders[new_price] = count_long
                        orders_long.append(new_price)
                        count_long = 0

            if orders_short:
                for num_order in range(len(orders_short)-1, -1, -1):
                    diff = calc_diff(ask, orders_short[num_order])
                    if abs(diff) >= close_diff and diff < 0:
                        body_order = tinvest.schemas.MarketOrderRequest(lots=dict_short_orders.get(orders_short[num_order]), operation=tinvest.schemas.OperationType.buy)
                        try:
                            if len(orders_short) != count and martingail != 0:
                                clots = orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order).json()['payload']['executedLots']
                                count -= findCount(martingail, count_lots_short, clots)
                            else:
                                count -= orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order).json()['payload']['executedLots'] / (count_lots_short + int(martingail*(count - 1)))
                        except Exception as e:
                            print('Exception Short close: {}'.format(e.args))
                            time.sleep(timer)
                            continue

                        oborot += dict_short_orders.get(orders_short[num_order]) * ask
                        marga += orders_short[num_order] * dict_short_orders.get(orders_short[num_order]) - ask * dict_short_orders.get(orders_short[num_order])
                        marga -= dict_short_orders.get(orders_short[num_order]) * ask * 0.0005 if oborot <= 200000 else dict_short_orders.get(orders_short[num_order]) * ask * 0.00025

                        print('Price: {} | Закрываем {} ШОРТ: {} | Всего заявок: {} | Оборотка: {} | Профит: {}'.format(ask, dict_short_orders.get(orders_short[num_order]), orders_short[num_order], count, oborot, marga))
                        print()
                        orders_short.remove(orders_short[num_order])
            if orders_long:
                for num_order in range(len(orders_long)-1, -1, -1):
                    diff = calc_diff(bid, orders_long[num_order])
                    if abs(diff) >= close_diff and diff > 0:
                        body_order = tinvest.schemas.MarketOrderRequest(lots=dict_long_orders.get(orders_long[num_order]), operation=tinvest.schemas.OperationType.sell)
                        try:
                            if len(orders_long) != count and martingail != 0:
                                clots = orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order).json()['payload']['executedLots']
                                count -= findCount(martingail, count_lots_long, clots)
                            else:
                                count -= orders_api.orders_market_order_post(figi=figi_rrc, broker_account_id=account_id, body=body_order).json()['payload']['executedLots'] / (count_lots_long + int(martingail*(count - 1)))
                        except Exception as e:
                            print('Exception Long close: {}'.format(e.args))
                            time.sleep(timer)
                            continue

                        oborot += dict_long_orders.get(orders_long[num_order]) * bid
                        marga += bid * dict_long_orders.get(orders_long[num_order]) - orders_long[num_order] * dict_long_orders.get(orders_long[num_order])
                        marga -= dict_long_orders.get(orders_long[num_order]) * bid * 0.0005 if oborot <= 200000 else dict_long_orders.get(orders_long[num_order]) * bid * 0.00025

                        print('Price: {} | Закрываем {} ЛОНГ: {} | Всего заявок: {} | Оборотка: {} | Профит: {}'.format(bid, dict_long_orders.get(orders_long[num_order]), orders_long[num_order], count, oborot, marga))
                        print()
                        orders_long.remove(orders_long[num_order])
        
        time.sleep(timer)
except Exception as e:
    print(e.args)

Прокомпилировать нижние блоки для выводы информации об оставшихся сделках

In [ ]:
count

In [ ]:
orders_short # цены, по которым открывали шорты

In [ ]:
orders_long # цены, по которым открывали лонги

In [ ]:
dict_short_orders # количество соответствующих открытых шортов

In [ ]:
dict_long_orders # количество соответствующих открытых лонгов